| ![EEW logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/eew.jpg?raw=true) | ![EDGI logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/edgi.png?raw=true) |
|---|---|

#### This notebook is licensed under GPL 3.0. Please visit our Github repo for more information: 
#### The notebook was collaboratively authored by the Environmental Data & Governance Initiative (EDGI) following our authorship protocol: https://docs.google.com/document/d/1CtDN5ZZ4Zv70fHiBTmWkDJ9mswEipX6eCYrwicP66Xw/
#### For more information about this project, visit https://www.environmentalenforcementwatch.org/

## How to Run this Notebook
* If you click on a gray **code** cell, a little “play button” arrow appears on the left. If you click the play button, it will run the code in that cell (“**running** a cell”). The button will animate. When the animation stops, the cell has finished running.
![Where to click to run the cell](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/pressplay.JPG?raw=true)
* You may get a warning that the notebook was not authored by Google. We know, we authored them! It’s okay. Click “Run Anyway” to continue. 
![Error Message](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/warning-message.JPG?raw=true)
* Run all of the cells in a Notebook to make a complete report. Please feel free to look at and **learn about each result as you create it**!

---

# Nationwide statistics about environmental compliance trends

## Setup
Here we load some helper code to get us going.

In [38]:
# Import code libraries
!git clone https://github.com/edgi-govdata-archiving/ECHO_modules.git &>/dev/null;
!git clone https://github.com/edgi-govdata-archiving/ECHO-COVID19.git &>/dev/null;
%run ECHO-COVID19/utilities.py
%run ECHO_modules/DataSet.py

import urllib.parse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import requests
import csv
import datetime
import folium
from folium.plugins import FastMarkerCluster
import ipywidgets as widgets
from IPython.core.display import display, HTML
from pandas.errors import EmptyDataError
def formatter(value):
  return "{:0.0f}".format(value)

Here we set up some code to help us store and eventually export the metrics.

In [41]:
inspections = dict()
violations = dict()
enforcements = dict()
penalties = dict()
emissions = dict()

## Start getting data
First, get summary data from the ECHO_EXPORTER table.

In [3]:
# Get everything we will need from ECHO_EXPORTER in a single DB query.
# We can then use the full dataframe to specialize views of it.
full_echo_data = None
column_mapping = {
    '"REGISTRY_ID"': str,
    '"FAC_NAME"': str,
    '"FAC_LAT"': float,
    '"FAC_LONG"': float,
    '"FAC_PERCENT_MINORITY"': float,
    '"AIR_IDS"': str,
    '"NPDES_IDS"': str,
    '"RCRA_IDS"': str,
    '"DFR_URL"': str,
    '"AIR_FLAG"': str,
    '"NPDES_FLAG"': str,
    '"GHG_FLAG"': str,
    '"RCRA_FLAG"': str,
    '"FAC_ACTIVE_FLAG"': str
}
column_names = list( column_mapping.keys() )
columns_string = ','.join( column_names )
sql = 'select ' + columns_string + ' from "ECHO_EXPORTER" where "AIR_FLAG" = \'Y\' or "NPDES_FLAG" = \'Y\' or "GHG_FLAG" = \'Y\' or "RCRA_FLAG" = \'Y\''
try:
    # Don't index.
    full_echo_data = get_data( sql )
except EmptyDataError:
    print("\nThere are no EPA facilities for this query.\n")
full_echo_data

,REGISTRY_ID,FAC_NAME,FAC_LAT,FAC_LONG,FAC_PERCENT_MINORITY,AIR_IDS,NPDES_IDS,RCRA_IDS,DFR_URL,AIR_FLAG,NPDES_FLAG,GHG_FLAG,RCRA_FLAG,FAC_ACTIVE_FLAG
0,1.100079e+11,FRANKLINVILLE CENTRAL OFFICE,39.619710,-75.082060,16.056,NaN,NaN,NJD980645550,http://echo.epa.gov/detailed-facility-report?f...,N,N,N,Y,NaN
1,1.100042e+11,INSTRUMENTAL ENGINEERING,41.023599,-74.202748,11.588,NaN,NaN,NJD089751200,http://echo.epa.gov/detailed-facility-report?f...,N,N,N,Y,NaN
2,1.100047e+11,MIKE BASS USED CAR,41.451417,-82.166056,46.117,NaN,NaN,OHD987049517,http://echo.epa.gov/detailed-facility-report?f...,N,N,N,Y,NaN
3,1.100706e+11,PALM PLAZA,28.818830,-81.887510,36.786,NaN,FLR10SZ00,NaN,http://echo.epa.gov/detailed-facility-report?f...,N,Y,N,N,Y
4,1.100202e+11,WILDLIFE RECREATION POND 1,32.247591,-87.791091,NaN,NaN,ALR165713,NaN,http://echo.epa.gov/detailed-facility-report?f...,N,Y,N,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1852340,1.100704e+11,PARKS AT LEGACY PHASE 2,33.229586,-96.850452,39.315,NaN,TXR15900B,NaN,http://echo.epa.gov/detailed-facility-report?f...,N,Y,N,N,Y
1852341,1.100052e+11,BERNHARDT & TAYLOR CUSTOM CABINETS,37.502910,-77.601800,24.065,NaN,NaN,VAD003117561,http://echo.epa.gov/detailed-facility-report?f...,N,N,N,Y,Y
1852342,1.100556e+11,FOUNDATION ENERGY MGMT - SCHRAMM #1,40.077161,-102.593594,37.500,CO0000000812502009,NaN,NaN,http://echo.epa.gov/detailed-facility-report?f...,Y,N,N,N,NaN
1852343,1.100201e+11,FLOYD PIT,31.771450,-86.007790,37.759,NaN,ALG890136 ALR161820,NaN,http://echo.epa.gov/detailed-facility-report?f...,N,Y,N,N,Y


## Number of Regulated Facilities in 2019 Per Program

In [23]:
air_fac = full_echo_data.loc[(full_echo_data["AIR_FLAG"]=="Y") & (full_echo_data["FAC_ACTIVE_FLAG"]=="Y")].shape[0]
water_fac = full_echo_data.loc[(full_echo_data["NPDES_FLAG"]=="Y") & (full_echo_data["FAC_ACTIVE_FLAG"]=="Y")].shape[0]
waste_fac = full_echo_data.loc[(full_echo_data["RCRA_FLAG"]=="Y") & (full_echo_data["FAC_ACTIVE_FLAG"]=="Y")].shape[0]
ghg_fac = full_echo_data.loc[(full_echo_data["GHG_FLAG"]=="Y") & (full_echo_data["FAC_ACTIVE_FLAG"]=="Y")].shape[0]

display(HTML("<h3>There are "+ str(air_fac) + " facilities currently regulated under the Clean Air Act.</h3>"))
display(HTML("<h3>There are "+ str(water_fac) + " facilities currently regulated under the Clean Water Act.</h3>"))
display(HTML("<h3>There are "+ str(waste_fac) + " facilities currently regulated under RCRA (hazardous waste).</h3>"))
display(HTML("<h3>There are "+ str(ghg_fac) + " facilities currently reporting greenhouse gas emissions.</h3>"))

## Clean Air Act inspections in 2019

In [4]:
# Use SQL to search for and select the data about air stack tests
air_inspections = None
try:
    sql = 'select * from \"ICIS-AIR_FCES_PCES\" where \"ACTUAL_END_DATE\" like \'__-__-20__\''

    # Download the data from that URL
    air_inspections = get_data( sql, 'pgm_sys_id' )
except EmptyDataError:
    print( "No data found")

air_inspections

,PGM_SYS_ID,ACTIVITY_ID,STATE_EPA_FLAG,ACTIVITY_TYPE_CODE,ACTIVITY_TYPE_DESC,COMP_MONITOR_TYPE_CODE,COMP_MONITOR_TYPE_DESC,ACTUAL_END_DATE,PROGRAM_CODES
0,020000003400546143,3400560126,S,INS,Inspection/Evaluation,FOO,FCE On-Site,12-01-2010,CAASIP
1,020000003400580001,3400566654,E,INS,Inspection/Evaluation,FOO,FCE On-Site,12-17-2002,CAASIP
2,020000003400710001,3400556492,E,INS,Inspection/Evaluation,PCE,PCE On-Site,07-13-2001,CAAMACT
3,020000003400710001,3600132199,S,INS,Inspection/Evaluation,FOO,FCE On-Site,07-08-2015,CAASIP
4,020000003400780001,3400559754,E,INS,Inspection/Evaluation,FOO,FCE On-Site,12-27-2002,CAASIP
...,...,...,...,...,...,...,...,...,...
1249716,VA0000005108900035,3401035767,S,INS,Inspection/Evaluation,PFF,PCE Off-Site,05-29-2007,"CAASIP, CAATVP"
1249717,VA0000005108900035,3401035768,S,INS,Inspection/Evaluation,PFF,PCE Off-Site,09-06-2007,"CAAMACT, CAASIP, CAATVP"
1249718,010000000901110001,3601047975,E,INS,Inspection/Evaluation,PFF,PCE Off-Site,03-10-2017,NaN
1249719,IL000161065AAW,3601561311,E,INS,Inspection/Evaluation,PCE,PCE On-Site,09-10-2018,NaN


In [145]:
# Number of facilities inspected each year per 1000 regulated facilities

#air_inspections["ACTUAL_END_DATE"] = pd.to_datetime(air_inspections["ACTUAL_END_DATE"], format='%m-%d-%Y', errors='coerce') # Format the date
air_inspections_metric = air_inspections.groupby(["ACTUAL_END_DATE"])[["STATE_EPA_FLAG"]].agg("count") # Count inspections per day
air_inspections_metric = air_inspections_metric.resample("Y").sum() # Sum up inspections per year
air_inspections_metric.index = air_inspections_metric.index.strftime('%Y') # Reformat the year (2016-12-31 becomes 2016)
air_inspections_metric_2019 = air_inspections_metric.loc["2019"] #Filter to 2019
air_inspections_metric_2019 = formatter((air_inspections_metric_2019["STATE_EPA_FLAG"] / air_fac) * 1000) #Divide by regulated facilities and multiply by desired rate (per 1000)
inspections["CAA"] = air_inspections_metric_2019
display(HTML("<h3>"+air_inspections_metric_2019+" inspections per 1000 facilities</h3>"))

## High priority violations of the Clean Air Act in 2019



In [44]:
air_violations = None
try:
    sql = 'select * from "ICIS-AIR_VIOLATION_HISTORY" where "HPV_DAYZERO_DATE" like \'__-__-20__\''

    air_violations = get_data( sql, "pgm_sys_id" )

    # Remove "FACIL" violations, which are paperwork violations according to: https://19january2017snapshot.epa.gov/sites/production/files/2013-10/documents/frvmemo.pdf
    # air_violations = air_violations.loc[(air_violations["POLLUTANT_DESCS"]!="FACIL")]
except EmptyDataError:
    print( "No data found")
air_violations

,PGM_SYS_ID,ACTIVITY_ID,AGENCY_TYPE_DESC,STATE_CODE,AIR_LCON_CODE,COMP_DETERMINATION_UID,ENF_RESPONSE_POLICY_CODE,PROGRAM_CODES,PROGRAM_DESCS,POLLUTANT_CODES,POLLUTANT_DESCS,EARLIEST_FRV_DETERM_DATE,HPV_DAYZERO_DATE,HPV_RESOLVED_DATE
0,AR0000000513900037,3400362735,State,AR,NaN,AR000A0000051390003700134,HPV,CAANSPS CAASIP CAATVP,New Source Performance Standards State Impleme...,300000243,VOLATILE ORGANIC COMPOUNDS (VOCS),NaN,05-18-2000,12-13-2002
1,AR0000000513900037,3400362739,State,AR,NaN,AR000A0000051390003700161,HPV,CAASIP CAATVP,State Implementation Plan for National Primary...,NaN,NaN,NaN,10-24-2001,06-19-2002
2,AR0000000513900037,3400362741,State,AR,NaN,AR000A0000051390003700216,HPV,CAANSPS CAASIP CAATVP,New Source Performance Standards State Impleme...,NaN,NaN,NaN,09-04-2004,08-11-2009
3,CASCA0000605900006,3400402885,Local,CA,SCA,CASCAA0000060590000600070,HPV,CAASIP CAATVP,State Implementation Plan for National Primary...,300000329,FACIL,NaN,06-10-2009,10-01-2009
4,CASCA0000605900006,3400402890,Local,CA,SCA,CASCAA0000060590000600095,HPV,CAASIP CAATVP,State Implementation Plan for National Primary...,300000329,FACIL,NaN,05-17-2012,08-01-2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30209,TX0000004820101886,3602092395,State,TX,NaN,TX000A0898324592011292002,HPV,CAATVP,Title V Permits,300000329,FACIL,10-05-2011,10-05-2011,11-16-2012
30210,TX0000004820101886,3602092396,State,TX,NaN,TX000A0899513422016068002,HPV,CAATVP,Title V Permits,300000329,FACIL,02-29-2016,02-29-2016,NaN
30211,TX0000004820101886,3602092404,State,TX,NaN,TX000A0995348982014072002,HPV,CAATVP,Title V Permits,300000329,FACIL,03-07-2014,03-14-2014,01-08-2015
30212,TX0000004835300036,3602092357,State,TX,NaN,TX000A0382202622003219002,HPV,CAATVP,Title V Permits,300000329,FACIL,07-06-2001,07-06-2001,07-01-2002


In [144]:
# Number of facility violations each year per 1000 regulated facilities

air_violations["HPV_DAYZERO_DATE"] = pd.to_datetime(air_violations["HPV_DAYZERO_DATE"], format='%m-%d-%Y', errors='coerce') # Format the date
air_violations_metric = air_violations.groupby(["HPV_DAYZERO_DATE"])[["AGENCY_TYPE_DESC"]].agg("count") # Count violations per day
air_violations_metric = air_violations_metric.resample("Y").sum() # Sum up violations per year
air_violations_metric.index = air_violations_metric.index.strftime('%Y') # Reformat the year (2016-12-31 becomes 2016)
air_violations_metric_2019 = air_violations_metric.loc["2019"] #Filter to 2019
air_violations_metric_2019 = formatter((air_violations_metric_2019["AGENCY_TYPE_DESC"] / air_fac) * 1000) #Divide by regulated facilities and multiply by desired rate (per 1000)
violations["CAA"] = air_violations_metric_2019
display(HTML("<h3>"+air_violations_metric_2019+" violations per 1000 facilities</h3>"))

## Enforcement Actions and Penalties under the Clean Air Act in 2019

In [50]:
air_enforcements = None
try:
    sql = 'select * from "ICIS-AIR_FORMAL_ACTIONS" where "SETTLEMENT_ENTERED_DATE" like \'__/__/20__\''

    air_enforcements = get_data( sql, "pgm_sys_id" )
except EmptyDataError:
    print( "No data found")
air_enforcements

,PGM_SYS_ID,ACTIVITY_ID,ENF_IDENTIFIER,ACTIVITY_TYPE_CODE,ACTIVITY_TYPE_DESC,STATE_EPA_FLAG,ENF_TYPE_CODE,ENF_TYPE_DESC,SETTLEMENT_ENTERED_DATE,PENALTY_AMOUNT
0,RI0000004400300502,1673,01-2002-0035,AFR,Administrative - Formal,E,113A,CAA 113A Admin Compliance Order (Non-Penalty),02/21/2002,0.0
1,020000003604700968,6686,02-1998-0336,JDC,Judicial,E,CIV,Civil Judicial Action,08/07/2000,1000000.0
2,020000003604700968,6686,02-1998-0336,JDC,Judicial,E,CIV,Civil Judicial Action,04/02/2007,0.0
3,AL0000000111700005,23504,04-2000-0099,JDC,Judicial,E,CIV,Civil Judicial Action,06/19/2006,100000.0
4,NY0000002630700276,8170,02-2001-0019,JDC,Judicial,E,BNK,Bankruptcy,04/20/2009,140625.0
...,...,...,...,...,...,...,...,...,...,...
82780,CO0000000812309009,3602243072,CO000A0000081230900900048,AFR,Administrative - Formal,S,SCAAAO,Administrative Order,07/07/2020,10500.0
82781,CO0000000801301467,3602243073,CO000A0000080130146700004,AFR,Administrative - Formal,S,SCAAAO,Administrative Order,06/16/2020,0.0
82782,CO0000000800102074,3602243074,CO000A0000080010207400004,AFR,Administrative - Formal,S,SCAAAO,Administrative Order,06/16/2020,0.0
82783,CO0000000800501688,3602243075,CO000A0000080050168800005,AFR,Administrative - Formal,S,SCAAAO,Administrative Order,06/16/2020,9100.0


In [143]:
# Number of actions each year per 1000 regulated facilities

air_enforcements["SETTLEMENT_ENTERED_DATE"] = pd.to_datetime(air_enforcements["SETTLEMENT_ENTERED_DATE"], format='%m/%d/%Y', errors='coerce') # Format the date
air_enforcements_metric = air_enforcements.groupby(["SETTLEMENT_ENTERED_DATE"])[["STATE_EPA_FLAG"]].agg("count") # Count enforcements per day
air_enforcements_metric = air_enforcements_metric.resample("Y").sum() # Sum up enforcements per year
air_enforcements_metric.index = air_enforcements_metric.index.strftime('%Y') # Reformat the year (2016-12-31 becomes 2016)
air_enforcements_metric_2019 = air_enforcements_metric.loc["2019"] #Filter to 2019
air_enforcements_metric_2019 = formatter((air_enforcements_metric_2019["STATE_EPA_FLAG"] / air_fac) * 1000) #Divide by regulated facilities and multiply by desired rate (per 1000)
enforcements["CAA"] = air_enforcements_metric_2019
display(HTML("<h3>"+air_enforcements_metric_2019+" enforcements per 1000 facilities</h3>"))

In [142]:
# Penalties each year per facility
air_penalties_metric = air_enforcements.loc[air_enforcements["PENALTY_AMOUNT"]>0]
air_penalties_metric = air_penalties_metric.groupby(["SETTLEMENT_ENTERED_DATE"])[["PENALTY_AMOUNT"]].agg("sum") # sum penalties per day
air_penalties_metric = air_penalties_metric.resample("Y").sum() # Sum up penalties per year
air_penalties_metric.index = air_penalties_metric.index.strftime('%Y') # Reformat the year (2016-12-31 becomes 2016)
air_penalties_metric_2019 = air_penalties_metric.loc["2019"] #Filter to 2019
air_penalties_metric_2019 = formatter(air_penalties_metric_2019["PENALTY_AMOUNT"] / air_fac) #Divide by regulated facilities
penalties["CAA"] = air_penalties_metric_2019
display(HTML("<h3>$"+air_penalties_metric_2019+" per facility</h3>"))

---

## Clean Water Act inspections in 2019

In [58]:
# Find facilities with pollutant exceedences
water_inspections = None
try:
    sql = 'select "NPDES_ID", "REGISTRY_ID", "ACTUAL_END_DATE", "STATE_EPA_FLAG"' + \
        ' from "NPDES_INSPECTIONS" where "ACTUAL_END_DATE" like \'__/__/20__%\''

    water_inspections = get_data( sql, "NPDES_ID" ) 
except EmptyDataError:
    print( "No data found")
water_inspections

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,REGISTRY_ID,ACTUAL_END_DATE,STATE_EPA_FLAG
NPDES_ID,,,
AK0000272,110000507693,08/21/2014,S
AK0000272,110000507693,09/22/2016,S
AK0000507,110030488620,02/22/2001,S
AK0000507,110030488620,12/17/2019,S
AK0000841,110007919547,07/20/2004,S
...,...,...,...
WYR320788,110060264173,04/17/2018,S
WYR320871,110070235311,03/25/2019,S
WYU000017,110070081440,05/22/2017,E


In [141]:
# Number of actions each year per 1000 regulated facilities

water_inspections["ACTUAL_END_DATE"] = pd.to_datetime(water_inspections["ACTUAL_END_DATE"], format='%m/%d/%Y', errors='coerce') # Format the date
water_inspections_metric = water_inspections.groupby(["ACTUAL_END_DATE"])[["STATE_EPA_FLAG"]].agg("count") # Count enforcements per day
water_inspections_metric = water_inspections_metric.resample("Y").sum() # Sum up enforcements per year
water_inspections_metric.index = water_inspections_metric.index.strftime('%Y') # Reformat the year (2016-12-31 becomes 2016)
water_inspections_metric_2019 = water_inspections_metric.loc["2019"] #Filter to 2019
water_inspections_metric_2019 = formatter((water_inspections_metric_2019["STATE_EPA_FLAG"] / water_fac) * 1000) #Divide by regulated facilities and multiply by desired rate (per 1000)
inspections["CWA"] = water_inspections_metric_2019
display(HTML("<h3>"+water_inspections_metric_2019+" inspections per 1000 facilities</h3>"))

## Effluent violations of the Clean Water Act in 2019
*NOTE*: Not other kind of violations (schedule, permit, single event)

In [61]:
# Find facilities with pollutant exceedences
water_violations = None
try:
    sql = 'select "NPDES_ID", "EXCEEDENCE_PCT", "MONITORING_PERIOD_END_DATE", "PARAMETER_DESC"' + \
        ' from "NPDES_EFF_VIOLATIONS" where "EXCEEDENCE_PCT" > 0 and "MONITORING_PERIOD_END_DATE" like \'__/__/20__%\''

    water_violations = get_data( sql, "NPDES_ID" ) 
except EmptyDataError:
    print( "No data found")
water_violations

,EXCEEDENCE_PCT,MONITORING_PERIOD_END_DATE,PARAMETER_DESC
NPDES_ID,,,
OH0045047,22,06/30/2003,"Solids, total suspended"
IN0041157,54,12/31/2003,"Nitrogen, ammonia total [as N]"
FL0037711,99999,09/30/2003,"Cadmium, total recoverable"
TX0022578,57,11/30/2004,"Solids, total suspended"
FL0041530,99999,02/29/2004,LC50 Statre 96Hr Acucyprinella Leedsi
...,...,...,...
RI0100072,52,09/30/2000,"Flow, in conduit or thru treatment plant"
MA0030066,300,03/31/2004,"Lead, total [as Pb]"
TN0020737,17,11/30/2004,"Solids, total suspended"


In [140]:
# Number of violations each year per 1000 regulated facilities

water_violations["MONITORING_PERIOD_END_DATE"] = pd.to_datetime(water_violations["MONITORING_PERIOD_END_DATE"], format='%m/%d/%Y', errors='coerce') # Format the date
water_violations_metric = water_violations.groupby(["MONITORING_PERIOD_END_DATE"])[["PARAMETER_DESC"]].agg("count") # Count per day
water_violations_metric = water_violations_metric.resample("Y").sum() # Sum up per year
water_violations_metric.index = water_violations_metric.index.strftime('%Y') # Reformat the year (2016-12-31 becomes 2016)
water_violations_metric_2019 = water_violations_metric.loc["2019"] #Filter to 2019
water_violations_metric_2019 = formatter((water_violations_metric_2019["PARAMETER_DESC"] / water_fac) * 1000) #Divide by regulated facilities and multiply by desired rate (per 1000)
violations["CWA"] = water_violations_metric_2019
display(HTML("<h3>"+water_violations_metric_2019+" violations per 1000 facilities</h3>"))

## Enforcement Actions and Penalties under the Clean Water Act in 2019

In [71]:
# Find facilities with pollutant exceedences
water_enforcements = None
try:
    sql = 'select "NPDES_ID", "AGENCY", "ENF_TYPE_DESC", "SETTLEMENT_ENTERED_DATE", "FED_PENALTY_ASSESSED_AMT", "STATE_LOCAL_PENALTY_AMT"' + \
        ' from "NPDES_FORMAL_ENFORCEMENT_ACTIONS" where "SETTLEMENT_ENTERED_DATE" like \'__/__/20__%\''

    water_enforcements = get_data( sql, "NPDES_ID" ) 
except EmptyDataError:
    print( "No data found")
water_enforcements

,AGENCY,ENF_TYPE_DESC,SETTLEMENT_ENTERED_DATE,FED_PENALTY_ASSESSED_AMT,STATE_LOCAL_PENALTY_AMT
NPDES_ID,,,,,
AK0021547,EPA,CWA 309G2B AO For Class II Penalties,04/20/2006,NaN,NaN
AKR10BF13,EPA,CWA 309G2E AO For Class I Penalties - Storm Wa...,08/24/2005,4250.0,NaN
AKR10BF44,EPA,Civil Judicial Action,08/04/2010,440000.0,NaN
AKR10BT00,EPA,CWA 309G2E AO For Class I Penalties - Storm Wa...,11/05/2007,7600.0,NaN
AKR10FE94,State,State CWA Penalty AO,03/02/2016,NaN,1750.0
...,...,...,...,...,...
WVR109230,State,State CWA Non Penalty AO,03/16/2020,NaN,NaN
WVR110156,State,State CWA Non Penalty AO,03/16/2020,NaN,NaN
WVR110197,State,State CWA Non Penalty AO,03/16/2020,NaN,NaN


In [158]:
# Number of enforcement actions each year per 1000 regulated facilities

water_enforcements["SETTLEMENT_ENTERED_DATE"] = pd.to_datetime(water_enforcements["SETTLEMENT_ENTERED_DATE"], format='%m/%d/%Y', errors='coerce') # Format the date
water_enforcements_metric = water_enforcements.groupby(["SETTLEMENT_ENTERED_DATE"])[["AGENCY"]].agg("count") # Count per day
water_enforcements_metric = water_enforcements_metric.resample("Y").sum() # Sum up per year
water_enforcements_metric.index = water_enforcements_metric.index.strftime('%Y') # Reformat the year (2016-12-31 becomes 2016)
water_enforcements_metric_2019 = water_enforcements_metric.loc["2019"] #Filter to 2019
water_enforcements_metric_2019 = formatter((water_enforcements_metric_2019["AGENCY"] / water_fac) * 1000) #Divide by regulated facilities and multiply by desired rate (per 1000)
enforcements["CWA"] = water_enforcements_metric_2019
display(HTML("<h3>"+water_enforcements_metric_2019+" enforcement actions per 1000 facilities</h3>"))

In [139]:
# Penalties each year per facility
water_penalties_metric = water_enforcements.loc[water_enforcements["FED_PENALTY_ASSESSED_AMT"]>0]
water_penalties_metric = water_penalties_metric.groupby(["SETTLEMENT_ENTERED_DATE"])[["FED_PENALTY_ASSESSED_AMT"]].agg("sum") # sum penalties per day
water_penalties_metric = water_penalties_metric.resample("Y").sum() # Sum up penalties per year
water_penalties_metric.index = water_penalties_metric.index.strftime('%Y') # Reformat the year (2016-12-31 becomes 2016)
water_penalties_metric_2019 = water_penalties_metric.loc["2019"] #Filter to 2019
water_penalties_metric_2019 = formatter(water_penalties_metric_2019["FED_PENALTY_ASSESSED_AMT"] / water_fac) #Divide by regulated facilities
penalties["CWA"] = water_penalties_metric_2019
display(HTML("<h3>$"+water_penalties_metric_2019+" per facility</h3>"))

## RCRA inspections in 2019

In [77]:
# Find facilities with pollutant exceedences
waste_inspections = None
try:
    sql = 'select * from "RCRA_EVALUATIONS" where "EVALUATION_START_DATE" like \'__/__/20__%\''

    waste_inspections = get_data( sql, "ID_NUMBER" ) 
except EmptyDataError:
    print( "No data found")
waste_inspections

,ACTIVITY_LOCATION,EVALUATION_IDENTIFIER,EVALUATION_TYPE,EVALUATION_DESC,EVALUATION_AGENCY,EVALUATION_START_DATE,FOUND_VIOLATION
ID_NUMBER,,,,,,,
MN0000232793,MN,001,CEI,COMPLIANCE EVALUATION INSPECTION ON-SITE,S,10/15/2003,Y
MN0000239541,MN,001,CEI,COMPLIANCE EVALUATION INSPECTION ON-SITE,S,06/29/2005,N
MN0000239814,MN,001,CEI,COMPLIANCE EVALUATION INSPECTION ON-SITE,S,04/04/2006,N
MN0000242891,MN,001,CEI,COMPLIANCE EVALUATION INSPECTION ON-SITE,S,12/30/2003,N
MN0000243048,MN,001,CEI,COMPLIANCE EVALUATION INSPECTION ON-SITE,S,11/01/2004,Y
...,...,...,...,...,...,...,...
GAR000057604,GA,001,CEI,COMPLIANCE EVALUATION INSPECTION ON-SITE,S,01/13/2020,N
RID063917140,RI,001,FCI,FOCUSED COMPLIANCE INSPECTION,S,11/04/2019,N
NED007286198,NE,001,FRR,FINANCIAL RECORD REVIEW,S,01/17/2020,N


In [138]:
# Number of inspections each year per 1000 regulated facilities
waste_inspections["EVALUATION_START_DATE"] = pd.to_datetime(waste_inspections["EVALUATION_START_DATE"], format='%m/%d/%Y', errors='coerce') # Format the date
waste_inspections_metric = waste_inspections.groupby(["EVALUATION_START_DATE"])[["EVALUATION_AGENCY"]].agg("count") # Count per day
waste_inspections_metric = waste_inspections_metric.resample("Y").sum() # Sum up per year
waste_inspections_metric.index = waste_inspections_metric.index.strftime('%Y') # Reformat the year (2016-12-31 becomes 2016)
waste_inspections_metric_2019 = waste_inspections_metric.loc["2019"] #Filter to 2019
waste_inspections_metric_2019 = formatter((waste_inspections_metric_2019["EVALUATION_AGENCY"] / waste_fac) * 1000) #Divide by regulated facilities and multiply by desired rate (per 1000)
inspections["RCRA"] = waste_inspections_metric_2019
display(HTML("<h3>"+waste_inspections_metric_2019+" inspections per 1000 facilities</h3>"))

## Violations of RCRA in 2019

In [80]:
# Find facilities with pollutant exceedences
waste_violations = None
try:
    sql = 'select * from "RCRA_VIOLATIONS" where "DATE_VIOLATION_DETERMINED" like \'__/__/20__%\''

    waste_violations = get_data( sql, "ID_NUMBER" ) 
except EmptyDataError:
    print( "No data found")
waste_violations

,ACTIVITY_LOCATION,VIOLATION_TYPE,VIOLATION_TYPE_DESC,VIOL_DETERMINED_BY_AGENCY,DATE_VIOLATION_DETERMINED,ACTUAL_RTC_DATE,SCHEDULED_COMPLIANCE_DATE
ID_NUMBER,,,,,,,
KSD031422306,KS,262.A,Standards Applicable to Generators of HW: General,S,04/18/2000,01/30/2001,12/01/2000
MOP000002105,MO,262.C,Standards Applicable to Generators of HW: Pre-...,S,10/27/2015,10/27/2015,NaN
MOR000553040,MO,273.B,Standards for Universal Waste Management: Stan...,S,12/22/2015,02/11/2016,NaN
ALD000826958,AL,262.C,Standards Applicable to Generators of HW: Pre-...,S,09/19/2000,12/11/2000,NaN
MOR000553693,MO,279.C,Standards for Used Oil: Generators,S,11/02/2015,11/02/2015,NaN
...,...,...,...,...,...,...,...
WAH000026408,WA,264.B,Standards for Owners and Operators of HW TSDs:...,S,05/06/2015,02/11/2016,12/31/2015
FLR000119792,FL,279.F,Standards for Used Oi:l Processors and Re-Refi...,S,08/19/2015,10/29/2015,NaN
AKR000205872,AK,279.C,Standards for Used Oil: Generators,E,11/20/2015,11/20/2015,NaN


In [137]:
# Number of violations each year per 1000 regulated facilities
waste_violations["DATE_VIOLATION_DETERMINED"] = pd.to_datetime(waste_violations["DATE_VIOLATION_DETERMINED"], format='%m/%d/%Y', errors='coerce') # Format the date
waste_violations_metric = waste_violations.groupby(["DATE_VIOLATION_DETERMINED"])[["VIOL_DETERMINED_BY_AGENCY"]].agg("count") # Count per day
waste_violations_metric = waste_violations_metric.resample("Y").sum() # Sum up per year
waste_violations_metric.index = waste_violations_metric.index.strftime('%Y') # Reformat the year (2016-12-31 becomes 2016)
waste_violations_metric_2019 = waste_violations_metric.loc["2019"] #Filter to 2019
waste_violations_metric_2019 = formatter((waste_violations_metric_2019["VIOL_DETERMINED_BY_AGENCY"] / waste_fac) * 1000) #Divide by regulated facilities and multiply by desired rate (per 1000)
violations["RCRA"] = waste_violations_metric_2019
display(HTML("<h3>"+waste_violations_metric_2019+" violations per 1000 facilities</h3>"))

## Enforcement Actions and Penalties under RCRA in 2019

In [83]:
# Find facilities with pollutant exceedences
waste_enforcements = None
try:
    sql = 'select * from "RCRA_ENFORCEMENTS" where "ENFORCEMENT_ACTION_DATE" like \'__/__/20__%\''

    waste_enforcements = get_data( sql, "ID_NUMBER" ) 
except EmptyDataError:
    print( "No data found")
waste_enforcements

,ACTIVITY_LOCATION,ENFORCEMENT_IDENTIFIER,ENFORCEMENT_TYPE,ENFORCEMENT_DESC,ENFORCEMENT_AGENCY,ENFORCEMENT_ACTION_DATE,PMP_AMOUNT,FMP_AMOUNT,FSC_AMOUNT,SCR_AMOUNT
ID_NUMBER,,,,,,,,,,
NYD980648562,NY,001,HQ110,VERBAL INFORMAL,E,08/12/2015,NaN,NaN,NaN,NaN
TNR000042812,TN,001,TN126,WARNING LETTER,S,06/02/2017,NaN,NaN,NaN,NaN
TNR000030643,TN,001,TN126,WARNING LETTER,S,06/06/2017,NaN,NaN,NaN,NaN
MAD985299791,MA,001,HQ120,WRITTEN INFORMAL,S,05/31/2017,NaN,NaN,NaN,NaN
WAD009266511,WA,001,HQ120,WRITTEN INFORMAL,S,06/14/2017,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
OHR000102475,OH,005,HQ120,WRITTEN INFORMAL,S,09/20/2013,NaN,NaN,NaN,NaN
NYD980763833,NY,001,HQ120,WRITTEN INFORMAL,S,03/26/2014,NaN,NaN,NaN,NaN
MOR000555540,MO,2,HQ140,LETTER OF INTENT TO INITIATE ENFORCEMENT ACTION,S,08/17/2016,NaN,NaN,NaN,NaN


In [136]:
# Number of enforcement actions each year per 1000 regulated facilities
waste_enforcements["ENFORCEMENT_ACTION_DATE"] = pd.to_datetime(waste_enforcements["ENFORCEMENT_ACTION_DATE"], format='%m/%d/%Y', errors='coerce') # Format the date
waste_enforcements_metric = waste_enforcements.groupby(["ENFORCEMENT_ACTION_DATE"])[["ENFORCEMENT_AGENCY"]].agg("count") # Count per day
waste_enforcements_metric = waste_enforcements_metric.resample("Y").sum() # Sum up per year
waste_enforcements_metric.index = waste_enforcements_metric.index.strftime('%Y') # Reformat the year (2016-12-31 becomes 2016)
waste_enforcements_metric_2019 = waste_enforcements_metric.loc["2019"] #Filter to 2019
waste_enforcements_metric_2019 = formatter((waste_enforcements_metric_2019["ENFORCEMENT_AGENCY"] / waste_fac) * 1000) #Divide by regulated facilities and multiply by desired rate (per 1000)
enforcements["RCRA"] = waste_enforcements_metric_2019
display(HTML("<h3>"+waste_enforcements_metric_2019+" enforcement actions per 1000 facilities</h3>"))

In [135]:
# Penalties each year per facility
waste_penalties_metric = waste_enforcements.loc[waste_enforcements["FMP_AMOUNT"]>0]
waste_penalties_metric = waste_penalties_metric.groupby(["ENFORCEMENT_ACTION_DATE"])[["FMP_AMOUNT"]].agg("sum") # sum penalties per day
waste_penalties_metric = waste_penalties_metric.resample("Y").sum() # Sum up penalties per year
waste_penalties_metric.index = waste_penalties_metric.index.strftime('%Y') # Reformat the year (2016-12-31 becomes 2016)
waste_penalties_metric_2019 = waste_penalties_metric.loc["2019"] #Filter to 2019
waste_penalties_metric_2019 = formatter(waste_penalties_metric_2019["FMP_AMOUNT"] / waste_fac) #Divide by regulated facilities
penalties["RCRA"] = waste_penalties_metric_2019
display(HTML("<h3>$"+waste_penalties_metric_2019+" per facility</h3>"))

## Greenhouse Gas Emissions in 2018 (latest data available)

In [88]:
# Find facilities with pollutant exceedences
ghg_emissions = None
try:
    sql = 'select * from "POLL_RPT_COMBINED_EMISSIONS" where "PGM_SYS_ACRNM" = \'E-GGRT\''

    ghg_emissions = get_data( sql) 
except EmptyDataError:
    print( "No data found")
ghg_emissions

,REPORTING_YEAR,REGISTRY_ID,PGM_SYS_ACRNM,PGM_SYS_ID,POLLUTANT_NAME,ANNUAL_EMISSION,UNIT_OF_MEASURE,NEI_TYPE,NEI_HAP_VOC_FLAG
0,2018,110058380130,E-GGRT,1011276,Carbon dioxide,15200.200,MTCO2e,NaN,NaN
1,2015,110058380087,E-GGRT,1011274,Carbon dioxide,28523.100,MTCO2e,NaN,NaN
2,2015,110058380087,E-GGRT,1011274,Nitrous oxide,17.284,MTCO2e,NaN,NaN
3,2018,110058380130,E-GGRT,1011276,Methane,12045.250,MTCO2e,NaN,NaN
4,2015,110058380130,E-GGRT,1011276,Methane,14528.000,MTCO2e,NaN,NaN
...,...,...,...,...,...,...,...,...,...
193802,2018,110002071739,E-GGRT,1000202,Methane,1286.000,MTCO2e,NaN,NaN
193803,2017,110002071739,E-GGRT,1000202,Nitrous oxide,20.562,MTCO2e,NaN,NaN
193804,2013,110002071739,E-GGRT,1000202,Carbon dioxide,24896.000,MTCO2e,NaN,NaN
193805,2013,110002071739,E-GGRT,1000202,Nitrous oxide,14.900,MTCO2e,NaN,NaN


In [134]:
# Emissions each year per facility
ghg_emissions["REPORTING_YEAR"] = pd.to_datetime(ghg_emissions["REPORTING_YEAR"], format='%Y', errors='coerce') # Format the date
ghg_emissions_metric = ghg_emissions.groupby(["REPORTING_YEAR"])[["ANNUAL_EMISSION"]].agg("sum") # sum emissions per year
ghg_emissions_metric.index = ghg_emissions_metric.index.strftime('%Y') # Reformat the year (2016-12-31 becomes 2016)
ghg_emissions_metric_2018 = ghg_emissions_metric.loc["2018"] #Filter to 2018
ghg_emissions_metric_2018 = formatter(ghg_emissions_metric_2018["ANNUAL_EMISSION"] / ghg_fac) #Divide by regulated facilities
emissions["GHG"] = ghg_emissions_metric_2018
display(HTML("<h3>"+ghg_emissions_metric_2018+" MTCO2e (metric tons of carbon dioxide equivalent) emissions per reporting facility</h3>"))

# Data Export

In [166]:
data = [inspections,
violations,
enforcements,
penalties,
emissions]

units = ["2019 inspections per 1000 facilities",
"2019 violations per 1000 facilities",
"2019 enforcements per 1000 facilities",
"2019 penalties per facility",
"2018 emissions per facility"]

short_units = ["inspections-rate",
"violations-rate",
"enforcements-rate",
"penalties-rate",
"emissions-rate"]

for index, program in enumerate(data):
    # create dataframe
    #x=pd.DataFrame.from_dict(program)
    df = pd.DataFrame(program, index=[0]).T
    df = df.rename(columns={0: units[index]})
    filename= short_units[index]+"_All_national_4_080720.csv" #active-facilities_All_MA-CD4_3b_080620.csv
    df.to_csv(filename)
    print(df)

     2019 inspections per 1000 facilities
CAA                                   261
CWA                                   148
RCRA                                   38
     2019 violations per 1000 facilities
CAA                                    4
CWA                                  410
RCRA                                  24
     2019 enforcements per 1000 facilities
CAA                                     14
CWA                                     11
RCRA                                    12
     2019 penalties per facility
CAA                          357
CWA                           16
RCRA                          29
    2018 emissions per facility
GHG                      509858
